In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import os
import fastparquet
import warnings
import geopy
from geopy.point import Point
import time
from pandas.core.common import SettingWithCopyWarning
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta, date
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [35]:
# Load in occ data
occ_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/occupancy')
occ_data.id = occ_data.id.astype('str')
miami_example = occ_data[occ_data['id'] == '52364652.0']


MemoryError: Unable to allocate 2.67 GiB for an array with shape (3, 119618530) and data type int64

In [ ]:
miami_example_by_pull_time = miami_example.groupby(['id','pulled'])['available','min_nights','available_for_checkin','available_for_checkout','bookable'].mean().reset_index()
miami_example_by_pull_time_count = miami_example.groupby(['pulled','date'])['available','min_nights','available_for_checkin','available_for_checkout','bookable'].count().reset_index()

# Create traces
miami_example_grouped_fig = go.Figure()
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['available'],
                    mode='lines',
                    name='Available'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['available_for_checkin'],
                    mode='lines', 
                    name='Available for Check In'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['available_for_checkout'],
                    mode='lines', 
                    name='Available for Check Out'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['bookable'],
                    mode='lines', 
                    name='Bookable'))
miami_example_grouped_fig.update_layout(title_text="Occupancy Data by Date", xaxis_title='Date', yaxis_title='Availability')
filename = "newsletter_features/occupancy_data_line_fig_miami_example.png"
miami_example_grouped_fig.write_image(filename, engine='kaleido', width=2000, height=500)
miami_example_grouped_fig.show()

In [ ]:
# Look at occ data in head(10) statement above to figure out what delta calcs need to be made
"""
occ_trend['available_delta'] = occ_trend['available_y'] - occ_trend['available_x']
occ_trend['available_for_checkin_delta'] = occ_trend['available_for_checkin_y'] - occ_trend['available_for_checkin_x']
occ_trend['available_for_checkout_delta'] = occ_trend['available_for_checkout_y'] - occ_trend['available_for_checkout_x']
occ_trend['bookable_delta'] = occ_trend['bookable_y'] - occ_trend['bookable_x']
occ_trend['available_delta_pct'] = occ_trend['available_delta']/occ_trend['available_x']*100
occ_trend['available_for_checkin_delta_pct'] = occ_trend['available_for_checkin_delta']/occ_trend['available_for_checkin_x']*100
occ_trend['available_for_checkout_delta_pct'] = occ_trend['available_for_checkout_delta']/occ_trend['available_for_checkout_x']*100
occ_trend['bookable_delta_pct'] = occ_trend['bookable_delta']/occ_trend['bookable_x']*100

occ_data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
occ_data_w_listing_loc = occ_data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
occ_trend['id'] = occ_trend['id'].astype('float64')
occ_trend_wloc = occ_trend.merge(occ_data_w_listing_loc, how='inner', on='id')
"""

In [ ]:
"""
florida_occ_trends = occ_trend_wloc[occ_trend_wloc['State'] == 'FL']
florida_occ_trends_zip = florida_occ_trends.groupby('zipcode').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()
florida_occ_trends_city = florida_occ_trends.groupby('City').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()
""""""
florida_price_trends_fig = go.Figure()
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['service_fee'],
                    mode='lines',
                    name='Service Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['total_price'],
                    mode='lines', 
                    name='Total Price'))
florida_price_trends_fig.update_layout(title_text="Pricing Data by Pull Time", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/florida_price_trends_fig_june19.png"
florida_price_trends_fig.write_image(filename, engine='kaleido', width=2000, height=500)
florida_price_trends_fig.show()
""""""
import plotly.graph_objects as go

zips = florida_occ_trends_zip['zipcode']
vals = [florida_occ_trends_zip['available_delta_pct'], florida_occ_trends_zip['available_for_checkin_delta_pct'], florida_occ_trends_zip['available_for_checkout_delta_pct'], florida_occ_trends_zip['bookable_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(255,0,0)' if v < 0 else 'rgb(0,125,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>Zip Code</b>'], ['<b>Availability</b>'], ['<b>Check-in Availability</b>'], ['<b>Check-out Availability</b>'],['<b>Bookable</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [zips, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Occupancy Trends in Miami (%) (M/M)',
                   title_x=0.5, showlegend=False)
florda_occ_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_occ_trends_table_june19.png"
florda_occ_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_occ_trend_fig.show()

# Look to do one price value and look and price change by day of the week
"""

In [ ]:
fwd_looking_days = 28
last_day_from = pd.to_datetime('05/31/2022')
pull_time_current = pd.to_datetime('05/05/2022') # Would be set as date for "today" in the future
fwd_looking_date_current = pull_time_current + timedelta(days=fwd_looking_days)
pull_time_old = last_day_from - timedelta(days=7)
fwd_looking_date_old = pull_time_old + timedelta(days=fwd_looking_days)

occ_data_last30_current = occ_data[(occ_data['date'] > str(pull_time_current)) & (occ_data['date'] <= str(fwd_looking_date_current))]
occ_data_last30_old = occ_data[(occ_data['date'] > str(pull_time_old)) & (occ_data['date'] <= str(fwd_looking_date_old))]
print(occ_data_last30_current.shape)
print(occ_data_last30_old.shape)
occ_data_last30_current['date'] = pd.to_datetime(occ_data_last30_current['date'])
occ_data_last30_old['date'] = pd.to_datetime(occ_data_last30_old['date'])
occ_data_last30_current['day_of_week'] = occ_data_last30_current['date'].dt.dayofweek
occ_data_last30_old['day_of_week'] = occ_data_last30_old['date'].dt.dayofweek
occ_data_previous_pull_avgs = occ_data_last30_old.groupby(['id', 'day_of_week']).mean().reset_index()
print(occ_data_previous_pull_avgs.shape)
occ_data_current_pull_avgs = occ_data_last30_current.groupby(['id', 'day_of_week']).mean().reset_index()
print(occ_data_current_pull_avgs.shape)
occ_trend = occ_data_previous_pull_avgs.merge(occ_data_current_pull_avgs, how='inner', on='id')
print(occ_trend.shape)
occ_trend.to_csv('occ_trend_miami_for_table_weekly_july30.csv')

In [ ]:
occ_trend['available_delta_pct'] = (occ_trend['available_y'] - occ_trend['available_x']) * 100
occ_trend['available_for_checkin_delta_pct'] = (occ_trend['available_for_checkin_y'] - occ_trend['available_for_checkin_x']) * 100
occ_trend['available_for_checkout_delta_pct'] = (occ_trend['available_for_checkout_y'] - occ_trend['available_for_checkout_x']) * 100
occ_trend['bookable_delta_pct'] = (occ_trend['bookable_y'] - occ_trend['bookable_x']) * 100
data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
occ_trend['id'] = occ_trend['id'].astype('float64')
occ_trend_wloc = occ_trend.merge(data_w_listing_loc, how='inner', on='id')

In [ ]:
florida_occ_trends = occ_trend_wloc[occ_trend_wloc['State'] == 'FL']
florida_occ_trends_zip = florida_occ_trends.groupby('zipcode').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()

import plotly.graph_objects as go

zips = florida_occ_trends_zip['zipcode']
vals = [florida_occ_trends_zip['available_delta_pct'], florida_occ_trends_zip['available_for_checkin_delta_pct'], florida_occ_trends_zip['available_for_checkout_delta_pct'], florida_occ_trends_zip['bookable_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(0,125,0)' if v < 0 else 'rgb(255,0,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>Zip Code</b>'], ['<b>Availability</b>'], ['<b>Check-in Availability</b>'], ['<b>Check-out Availability</b>'],['<b>Bookable</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [zips, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Occupancy Trends in Miami (%) (W/W)',
                   title_x=0.5, showlegend=False)
florda_occ_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_occ_trends_table_weekly_july30.png"
florda_occ_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_occ_trend_fig.show()

# Look to do one price value and look and price change by day of the week

In [ ]:
florida_occ_trends_zip_propcounts = florida_occ_trends.groupby('zipcode').count().reset_index()
florida_occ_trends_zip_propcounts

In [ ]:
florida_occ_trends = occ_trend_wloc[occ_trend_wloc['State'] == 'FL']
florida_occ_trends_city = florida_occ_trends.groupby('City').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()

import plotly.graph_objects as go

cities = florida_occ_trends_city['City']
vals = [florida_occ_trends_city['available_delta_pct'], florida_occ_trends_city['available_for_checkin_delta_pct'], florida_occ_trends_city['available_for_checkout_delta_pct'], florida_occ_trends_city['bookable_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(0,125,0)' if v < 0 else 'rgb(255,0,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>City</b>'], ['<b>Availability</b>'], ['<b>Check-in Availability</b>'], ['<b>Check-out Availability</b>'],['<b>Bookable</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [cities, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Occupancy Trends in Miami (%) (W/W)',
                   title_x=0.5, showlegend=False)
florda_occ_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_occ_trends_table_city_weekly_july30.png"
florda_occ_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_occ_trend_fig.show()

# Look to do one price value and look and price change by day of the week

In [ ]:
import plotly.express as px
from urllib.request import urlopen
import json

with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/fl_florida_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)

florida_occ_trends = occ_trend_wloc[occ_trend_wloc['State'] == 'FL']
florida_occ_trends_zipformap = florida_occ_trends.groupby('zipcode').mean('available').reset_index()

fig = px.choropleth(florida_occ_trends_zipformap, 
                    geojson=zipcodes, 
                    locations='zipcode', 
                    color='available_delta_pct',
                    color_continuous_scale="RdYlGn",
                    featureidkey="properties.ZCTA5CE10",
                    range_color=(-100,100),
                    scope="usa"
                    #labels={'Price Change':'Cluster_Category'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
filename = "newsletter_features/florida_occ_trends_heatmap_zip_weekly_july30.png"
florda_occ_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
fig.show()

In [9]:
occ_trend = pd.read_csv('occ_trend_miami_for_table_weekly.csv')

In [10]:
occ_trend['available_delta_pct'] = (occ_trend['available_y'] - occ_trend['available_x']) * 100
occ_trend['available_for_checkin_delta_pct'] = (occ_trend['available_for_checkin_y'] - occ_trend['available_for_checkin_x']) * 100
occ_trend['available_for_checkout_delta_pct'] = (occ_trend['available_for_checkout_y'] - occ_trend['available_for_checkout_x']) * 100
occ_trend['bookable_delta_pct'] = (occ_trend['bookable_y'] - occ_trend['bookable_x']) * 100
data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
occ_trend['id'] = occ_trend['id'].astype('float64')
occ_trend_wloc = occ_trend.merge(data_w_listing_loc, how='inner', on='id')

In [11]:
occ_trend_wloc.to_csv('occ_trend_w_loc_miami.csv')

In [12]:
occ_trend_wloc_pvttbl = occ_trend_wloc.pivot_table(index=['zipcode','day_of_week_x'], values='available_delta_pct', aggfunc='mean').reset_index()
occ_trend_wloc_pvttbl.to_csv('occ_trend_wloc_pvttbl.csv')


In [ ]:
# Get Florida data for example viz
florida_occ_trends = occ_trend_wloc[occ_trend_wloc['State'] == 'FL']

# Pivot table on zipcode and day of the week
florida_occ_trends_zip_dow = florida_occ_trends.pivot_table(index=['zipcode','day_of_week_x'], values='available_delta_pct', aggfunc='mean')


florida_occ_trends_zip_dow_linechart = florida_occ_trends_zip_dow.reset_index()
fig_line_occ_zips = px.line(florida_occ_trends_zip_dow_linechart, x="day_of_week_x", y="available_delta_pct", color='zipcode',                  labels={
                     "day_of_week_x": "Day of Week",
                     "available_delta_pct": "Pct Occupancy Rate Change"
                 })
fig_line_occ_zips.update_layout(title='Occupancy Changes in Miami W/W Monday-Sunday')
fig.update_xaxes(range=[-100, 100])
filename = "newsletter_features/florida_occ_trends_lines_zip_weekly_july30.png"
florda_occ_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
fig_line_occ_zips.show()